# Aplicação prática do modelo de rotulação de Lopes(2014)

In [ ]:
import pandas as pd
import numpy as np

## Seleção de parâmetros

In [ ]:
variacao = 8.5
faixa_valores = 3
per_trein = 0.25
folds = 10

In [ ]:
grupo = "sementes.csv"

In [ ]:
grupo = "iris.csv"

In [ ]:
grupo = "vidros.csv"

In [ ]:
grupo = "wine.csv"

In [ ]:
grupo = "mnist64.csv"

In [ ]:
tipo_discretizacao = 'EWD'

In [ ]:
tipo_discretizacao = 'EFD'

In [ ]:
alg_super = 'PMC'

In [ ]:
alg_super = 'TREE'

In [ ]:
print(f'Parametros: bd={grupo}, disc={tipo_discretizacao}, algoritmo={alg_super}')
print(f'variacao={variacao}, faixa={faixa_valores}, % de treino={per_trein}, treinos e testes={folds}')

## Tratando os dados

In [ ]:
base = pd.read_csv(grupo)

In [ ]:
base.head()

In [ ]:
attr_classe = base.columns[-1]
print(f'atributo escolhido: {attr_classe}')

frames = []
for index, group in base.groupby(attr_classe):
    frames.append(group)
print(f'numero de clusters: {len(frames)}')

In [ ]:
frames[2].shape

In [ ]:
frames[0].head()

### Discretização

In [ ]:
import discretizacao as dsc

discData = dsc.discretizador(base, faixa_valores, tipo_discretizacao, attr_classe)
base_discretizada = discData.ddb
infor = discData.infor

In [ ]:
print(base_discretizada[:10])

In [ ]:
for info in infor:
    print(info)

In [ ]:
frames_disc = []
for index, group in base_discretizada.groupby(attr_classe):
    frames_disc.append(group)
print(f'numero de clusters: {len(frames_disc)}')

In [ ]:
frames_disc[0].head()

## Predição de atributos (Aprendizagem supervisionada)

In [ ]:
import supervised as sp

infor_attrs = sp.aciona_metodo(frames_disc, attr_classe, per_trein, folds, alg_super)

for info in infor_attrs:
    print(info)

## Rotulação

In [ ]:
import rotulacao as rot

rotulos = rot.atrib_rotul(frames_disc, attr_classe, infor_attrs, variacao, infor)

print("Rotulos")
for rotulo in rotulos:
    print(f"Cluster: {rotulo[0]}")
    print(f"Pares Atributo-Intervalo: {rotulo[1]}")
    print()

## Verificando rótulos

#### Testando cada rótulo com o grupo correspondente

In [ ]:
count = 0
for fra in frames:
    fr = fra.copy()
    print(rotulos[count])
    print(f'quantidade de dados na base original: {len(fra)}')
    for tupla in rotulos[count][1]:
        fr = fr[(fr[tupla[0]]>=tupla[1]) & (fr[tupla[0]]<=tupla[2])]
    print(f'quantidade de dados rotulados: {len(fr)}')
    print()
    count += 1  

#### Testando os rótulos com a base inteira

In [ ]:
count = 0
bd = []
for rotulo in rotulos:
    bd.append(base.copy())
    for tupla in rotulo[1]:
        bd[count] = bd[count][(bd[count][tupla[0]]>=tupla[1]) & (bd[count][tupla[0]]<=tupla[2])]
    print(f'data points designados ao cluster {count+1}: {len(bd[count])}')
    count+=1

bdd = bd[0].copy()
for b in range(len(bd)):
    if b == 0:
        continue
    bdd = bdd + bd[b]
print(f'data points que receberam algum rotulo: {len(bdd)}')